In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
import pandas as pd
import seaborn as sns
from IPython.display import display
from sklearn.compose import ColumnTransformer , make_column_transformer, TransformedTargetRegressor  
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score, KFold, StratifiedShuffleSplit, GridSearchCV
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer,PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder,QuantileTransformer,PowerTransformer
from sklearn.datasets import make_classification
from sklearn.metrics import precision_score, classification_report, confusion_matrix, ConfusionMatrixDisplay,log_loss
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,mean_absolute_percentage_error,f1_score,accuracy_score
import category_encoders as ce
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score, mean_absolute_percentage_error
from category_encoders import TargetEncoder
from sklearn.linear_model import ElasticNet, Ridge, Lasso, Lars,SGDRegressor,LogisticRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,GradientBoostingRegressor,RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer , make_column_transformer, TransformedTargetRegressor
from sklearn.covariance import EllipticEnvelope
from sklearn.svm import OneClassSVM
from sklearn import svm
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.datasets import make_classification




In [2]:
path_train = 'train_house_B.csv'

In [3]:
df = pd.read_csv(path_train)
display(df.shape)

(408, 30)

In [4]:
cor =df.corr()

In [5]:
features = cor[df.corr()>0.3]['price'].dropna().index
features

Index(['price', '2', '12', '18', '19', '20', '21', '22', '24', '25', '27'], dtype='object')

In [6]:
X = df.drop(columns=['price'])
y = (df.price)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42 )

In [7]:
selected_features = ['2', '12', '18', '19', '20', '21', '22', '24', '25', '27']

X_trainselected = X_train[selected_features]
X_testselected = X_test[selected_features]


In [8]:
ct = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), selected_features)],
      
    remainder='drop'  
).set_output(transform="pandas")

display(ct)

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['2', '12', '18', '19', '20', '21', '22', '24',
                                  '25', '27'])])

In [9]:
model_baseline_1 = Pipeline(steps = [
                          ('preproc',ct),
                        ('estimator',RandomForestRegressor(n_estimators =500, n_jobs = -1))
                        ])
    

model_baseline_1.fit(X_trainselected,y_train)

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['2', '12', '18', '19', '20',
                                                   '21', '22', '24', '25',
                                                   '27'])])),
                ('estimator',
                 RandomForestRegressor(n_estimators=500, n_jobs=-1))])

In [10]:
y_pred_test1 = model_baseline_1.predict(X_test)
mape_test1 = mean_absolute_percentage_error(y_test, y_pred_test1)
print(f'MAPE_TEST: {mape_test1}')

MAPE_TEST: 0.21840438899041767


In [11]:
data = ct.fit_transform(X_train)
data

,num__2,num__12,num__18,num__19,num__20,num__21,num__22,num__24,num__25,num__27
158,-1.398862,-1.453432,-0.240814,-0.148213,-0.554382,0.052408,1.378979,-0.761059,0.165592,-0.707107
109,0.675727,0.808443,-0.240814,1.416258,1.803810,0.052408,1.378979,-0.761059,1.316659,1.414214
17,-1.143778,-0.204715,-0.240814,-0.148213,-0.554382,-1.305443,-0.538138,0.374553,-0.985475,1.414214
237,-0.801808,-0.204715,-0.240814,-0.148213,-0.554382,-1.305443,-0.538138,0.374553,0.165592,-0.707107
24,-0.532320,-0.508662,1.777774,-0.148213,-0.554382,1.410260,1.378979,-0.761059,0.165592,-0.707107
...,...,...,...,...,...,...,...,...,...,...
71,0.071704,0.149891,1.777774,-1.712684,1.803810,1.410260,1.378979,-0.761059,-0.985475,1.414214
106,0.443410,0.555154,-0.240814,-1.712684,1.803810,0.052408,1.378979,1.510164,-0.985475,-0.707107
270,-0.063040,-0.204715,-0.240814,-0.148213,-0.554382,0.052408,-0.538138,-0.761059,0.165592,-0.707107
348,0.968911,-0.204715,-0.240814,-0.148213,1.803810,0.052408,-0.538138,-0.761059,0.165592,1.414214


In [12]:
forest = RandomForestRegressor(n_estimators =500,min_samples_split=5,n_jobs = -1,max_depth = 5,
                                                               random_state=42)
forest.fit(data,y_train)

RandomForestRegressor(max_depth=5, min_samples_split=5, n_estimators=500,
                      n_jobs=-1, random_state=42)

In [13]:
n_features = data.shape[1]
indices = np.argsort(forest.feature_importances_)[::-1]
sorted_importances = forest.feature_importances_[indices]
sorted_features = data.columns[indices]
sorted_features

Index(['num__12', 'num__2', 'num__25', 'num__24', 'num__20', 'num__22',
       'num__21', 'num__27', 'num__18', 'num__19'],
      dtype='object')

In [14]:
selected_features2 = [ '12', '2', '25', '24', '20', '22' , '21', '27','18','19']
X_trainselected2 = X_train[selected_features2]
X_testselected2 = X_test[selected_features2]

In [15]:
ct_1 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), selected_features2)],
      
    remainder='drop'  
).set_output(transform="pandas")

display(ct)

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['2', '12', '18', '19', '20', '21', '22', '24',
                                  '25', '27'])])

In [16]:
model = Pipeline(steps = [
                          ('preproc',ct_1),
                        ('estimator',Ridge())
                        ])

model.fit(X_trainselected2,y_train)

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['12', '2', '25', '24', '20',
                                                   '22', '21', '27', '18',
                                                   '19'])])),
                ('estimator', Ridge())])

In [17]:
y_pred_test = model.predict(X_test)
mape_test= mean_absolute_percentage_error(y_test, y_pred_test)
print(f'MAPE_TEST: {mape_test4}')

y_pred_train = model.predict(X_train)
mape_train = mean_absolute_percentage_error(y_train, y_pred_train)
print(f'MAPE_TRAIN: {mape_train4}')

NameError: name 'mape_test4' is not defined